<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [ ]:
# Set your student number
student_number = 98103867
Name = 'Mohammad'
Last_Name = 'Abolnejadian'

# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.3 | 0.5 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

<font size=4>
Author: Atoosa Chegini
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

In [51]:
import numpy as np
import random
import gym

In [67]:
env = gym.make("MountainCar-v0")

# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)
number_of_states = 50
max_alpha = 1.0
min_alpha = 0.005

iters = 500_000
steps = 10_000

gamma = 1.0

number_of_actions = 3

# You may change the inputs of any function as you desire.
q_table = np.zeros((number_of_states, number_of_states, number_of_actions))

Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [68]:
# This is just one example of a discretization function. You can change it as you want.
def discretize_state(x, minn, step):
    return int((x - minn) / step)


def env_state_to_Q_state(state):
    [position, velocity] = state
    pos_low, pos_high = -1.2, 0.6
    vel_low, vel_high =  -0.07, 0.07
    # Complete this function!
    pos_step = (pos_high - pos_low) / number_of_states
    vel_step = (vel_high - vel_low) / number_of_states

    p = discretize_state(position, pos_low, pos_step)
    v = discretize_state(velocity, vel_low, vel_step)

    return (p, v)


def update_q(current_state, next_state, action, reward, alpha):
    # Complete this function!

    best_future_q = np.max(q_table[next_state])
    sample = reward + gamma * best_future_q

    q_table[current_state][action] = (1 - alpha)*(q_table[current_state][action]) + (alpha)*(sample)

At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [81]:
# You may change the inputs of any function as you desire.
def get_action(current_state):
  return np.argmax(q_table[current_state])


def q_learning():
  # Complete this funciton!
  for i in range(iters):
    obs = env.reset()
    total_reward = 0
    # Learning rate decreasing as we move forward (formula based on a repo)
    decreasing_alpha = max(min_alpha, max_alpha * (0.85 ** (i//100)))

    for _ in range(steps):
      current_state = env_state_to_Q_state(obs)
      
      action = get_action(current_state)
      
      obs, reward, done, _ = env.step(action)
      total_reward += reward
      next_state = env_state_to_Q_state(obs)
      update_q(current_state, next_state, action, reward, decreasing_alpha)

      if done:
        break



def save_policy():
  policy  = []
  for i in range(number_of_states):
      policy.append([])
      for j in range(number_of_states):
          policy[i].append([])
          policy[i][j] = (np.argmax(q_table[i][j]))
  np.save('policy.npy', policy)

In [82]:
q_learning()
save_policy()

In [83]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
def score():
    policy, scores = np.load("policy.npy"), []
    print(policy)
    for episode in range(1000):
        print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(env.reset()), 0, False, 0
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            step += 1
            score += int(reward)
            # env.render()
        print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")

score()

[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
******Episode 0
Score:-116
******Episode 1
Score:-114
******Episode 2
Score:-116
******Episode 3
Score:-114
******Episode 4
Score:-117
******Episode 5
Score:-116
******Episode 6
Score:-115
******Episode 7
Score:-117
******Episode 8
Score:-115
******Episode 9
Score:-115
******Episode 10
Score:-116
******Episode 11
Score:-115
******Episode 12
Score:-114
******Episode 13
Score:-115
******Episode 14
Score:-117
******Episode 15
Score:-115
******Episode 16
Score:-113
******Episode 17
Score:-116
******Episode 18
Score:-117
******Episode 19
Score:-113
******Episode 20
Score:-110
******Episode 21
Score:-115
******Episode 22
Score:-112
******Episode 23
Score:-115
******Episode 24
Score:-116
******Episode 25
Score:-113
******Episode 26
Score:-120
******Episode 27
Score:-110
******Episode 28
Score:-111
******Episode 29
Score:-115
******Episode 30
Score:-116
******Episode 31
Score